In [1]:
import requests

In [ ]:
!pip install elasticsearch==8.15.1

In [2]:
from elasticsearch import Elasticsearch, RequestError
from elasticsearch.client import IndicesClient
from elasticsearch.exceptions import NotFoundError
from elasticsearch.helpers import parallel_bulk

/tmp/ipykernel_3616/4078056810.py:2: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import IndicesClient


In [3]:
es_hosts = ["http://140.96.83.134:32626"]
es = Elasticsearch(es_hosts, request_timeout=60*60)
es_indices = IndicesClient(es)

In [4]:
INDEXS = ['snomedct_us', 'rxnorm', 'lnc']

In [5]:
# 刪除原本的index

for index in INDEXS:
    try:
        print(f'delete index `{index}`...')
        es_indices.delete(index=index)
    except NotFoundError as e:
        # 若沒有該index, 則略過.
        print(f'index不存在不需要刪除')


delete index `snomedct_us`...
index不存在不需要刪除
delete index `rxnorm`...
index不存在不需要刪除
delete index `lnc`...
index不存在不需要刪除


In [6]:
# 增加schema
schema = {
  "settings": {
    "analysis": {
      "analyzer": {
        "my_index_analyzer": {
          "tokenizer": "my_simple_pattern_tokenizer",
          "filter": [
            "my_edgegram",
            "lowercase"
          ]
        },
        "my_search_analyzer": {
          "tokenizer": "my_simple_pattern_tokenizer",
          "filter": [
            "lowercase"
          ]
        }
      },
      "tokenizer": {
        "my_simple_pattern_tokenizer": {
          "type": "simple_pattern",
          "pattern": "(\\d+|[A-Za-z]+|[^\\s\\dA-Za-z\u0021-\u002F\u003A-\u0040\u005B-\u005F\u007B-\u007E\u00A1-\u00BF])"
        },
        "my_tokenizer": {
          "type": "edge_ngram",
          "min_gram": 2,
          "max_gram": 50,
          "token_chars": [
            "letter",
            "digit",
            "symbol",
            "punctuation"
          ]
        }
      },
      "filter": {
        "my_edgegram": {
          "type": "edge_ngram",
          "min_gram": 2,
          "max_gram": 50,
          "preserve_original": True
        }
      }
    }
  },
  "mappings": {
      "dynamic": "true",
      "properties": {
          "CUI": {
              "type": "keyword"
          },
          "SAB": {
              "type": "keyword"
          },
          "TTY": {
              "type": "keyword"
          },
          "CODE": {
            "type": "keyword"
          },
          "STR": {
              "type": "text",
              "analyzer": "my_index_analyzer",
              "search_analyzer":"my_search_analyzer"
          }
      }
  }
}

for index in INDEXS:
    print(f'add schema to index `{index}`...')
    try:
        es_indices.create(index=index, body=schema)
    except RequestError as e:
        logger.warning(e)
        logger.warning(str(traceback.format_exc()))
        logger.info(f'schema already exist (index=`{index}`).')

add schema to index `snomedct_us`...
add schema to index `rxnorm`...
add schema to index `lnc`...


In [7]:
import json
def doc_generator():
    for i in [1, 2, 3, 4]:
        with open(f'filtered_umls_{i}_with_op.ndjson') as f:
            while True:
                line1 = f.readline().strip()  # e.g. {"index": {"_index": "rxnorm"}}
                line2 = f.readline().strip()  # e.g. {"CUI": "C0000039", "SAB": "RXNORM", "TTY": "IN", "CODE": "1926948", "STR": "1,2-dipalmitoylphosphatidylcholine"}
    
                # 如果已經到達文件末尾，跳出迴圈
                if not line1 or not line2:
                    break
    
                yield  {
                    '_op_type': 'index',
                    '_index': json.loads(line1)['index']['_index'],
                    **json.loads(line2.strip())
                }

In [ ]:
# doc_generator 測試
for j, i in enumerate(doc_generator()):
    print(i)
    if j == 10:
        break

In [9]:
from tqdm import tqdm
fail_count = 0
for is_success, info in tqdm(parallel_bulk(es, doc_generator(), chunk_size=500, raise_on_error=True)):
    if is_success:
        pass
    else:
        print(f'文件索引失敗. 錯誤資訊={info}')
        fail_count += 1

1897402it [03:31, 8975.73it/s] 


In [10]:
fail_count

0